# Imports

In [1]:
from dotenv import load_dotenv

from saw.workflows.multi_llm.chaining import chain, achain
from saw.workflows.multi_llm.parallelization import parallel, aparallel
from saw.workflows.multi_llm.routing import route, aroute

In [2]:
load_dotenv("../docker/.env")

True

# Variables

In [3]:
provider = "google"
model = "gemini-2.0-flash"
provider2 = "ollama"
model2 = "deepseek-r1:1.5b"
provider3 = "groq"
model3 = "llama3-8b-8192"
provider4 = "openai"
model4 = "gpt-4o-mini"
system_prompt = "You are a helpful assistant."

# Chaining Example
The `chain` function allows us to process a query through multiple prompts sequentially.

In [4]:
query = "What is the best way to master deep learning which is very complex and technical?"

prompts = [
    {
        "prompt": "Provide an overview of the subjects needed for mastering deep learning.",
        "model": model,
        "provider": provider,
        "system_prompt": system_prompt,
        "functions": [str.lower]  # Example function to lower case the prompt
    },
    {
        "prompt": "Provide a step-by-step guide on how to learn these necessary topics so I can master deep learning.",
        "model": model,
        "provider": provider,
        "system_prompt": system_prompt,
        "functions": [str.lower]
    }
]

In [5]:
# Synchronous chaining
result = chain(query=query, prompts=prompts)

Model: google-gemini-2.0-flash

Step 1: provide an overview of the subjects needed for mastering deep learning.
Response: dict_keys(['candidates', 'create_time', 'response_id', 'model_version', 'prompt_feedback', 'usage_metadata', 'automatic_function_calling_history', 'parsed'])
Model: gemini-2.0-flash
Usage: cached_content_token_count=None candidates_token_count=1939 prompt_token_count=37 total_token_count=1976

Result: Okay, you're right, deep learning is a complex and technical field! Mastering it requires a good understanding of several core subjects and a commitment to hands-on practice. Here's an overview of the key subjects you'll need to focus on, broken down into categories:

**I. Foundational Mathematics:**

*   **Linear Algebra:** This is absolutely *essential*. Deep learning models rely heavily on matrix operations, vector spaces, and linear transformations. You need to understand:
    *   Vectors and Matrices:  Representations, operations (addition, multiplication, transpo

# Parallelization Example
The `parallel` function allows us to process multiple inputs in parallel using a model.

In [6]:
prompts = [
    {
        "prompt": "What is the best way to learn a complex technical concept in deep learning?",
        "model": model,
        "provider": provider,
        "system_prompt": system_prompt,
        "functions": [str.lower]
    },
    {
        "prompt": "What is the best way to learn a complex technical concept in quantum computing?",
        "model": model2,
        "provider": provider2,
        "system_prompt": system_prompt,
        "functions": [str.lower]
    },
    {
        "prompt": "What are the key differences between supervised and unsupervised learning?",
        "model": model,
        "provider": provider,
        "system_prompt": system_prompt,
        "functions": [str.lower]
    },
    {
        "prompt": "How can I get started with quantum computing as a beginner?",
        "model": model2,
        "provider": provider2,
        "system_prompt": system_prompt,
        "functions": [str.lower]
    }
]

n_workers = 4

In [7]:
# Synchronous parallelization
results = parallel(query=query, prompts=prompts, n_workers=n_workers)

Response: dict_keys(['candidates', 'create_time', 'response_id', 'model_version', 'prompt_feedback', 'usage_metadata', 'automatic_function_calling_history', 'parsed'])
Model: gemini-2.0-flash
Usage: cached_content_token_count=None candidates_token_count=856 prompt_token_count=36 total_token_count=892
Response: dict_keys(['candidates', 'create_time', 'response_id', 'model_version', 'prompt_feedback', 'usage_metadata', 'automatic_function_calling_history', 'parsed'])
Model: gemini-2.0-flash
Usage: cached_content_token_count=None candidates_token_count=1601 prompt_token_count=40 total_token_count=1641
Response: dict_keys(['model', 'created_at', 'done', 'done_reason', 'total_duration', 'load_duration', 'prompt_eval_count', 'prompt_eval_duration', 'eval_count', 'eval_duration', 'response', 'context'])
Using model: deepseek-r1:1.5b
Response: dict_keys(['model', 'created_at', 'done', 'done_reason', 'total_duration', 'load_duration', 'prompt_eval_count', 'prompt_eval_duration', 'eval_count', '

In [8]:
results[0]

('what is the best way to learn a complex technical concept in deep learning?',
 'Okay, tackling complex deep learning concepts requires a multi-faceted approach. Here\'s a breakdown of the best ways to learn them deeply:\n\n**1. Solid Foundation (Prerequisites):**\n\n*   **Mathematics:** This is non-negotiable.\n    *   **Linear Algebra:** Vectors, matrices, matrix operations, eigenvalues/eigenvectors, decompositions (SVD, PCA).  Understand how these are used to represent data and transformations within neural networks.\n    *   **Calculus:** Derivatives, gradients, chain rule. Essential for understanding backpropagation and optimization.\n    *   **Probability and Statistics:** Probability distributions, hypothesis testing, Bayesian inference.  Necessary for understanding model evaluation, uncertainty, and generative models.\n*   **Programming Fundamentals:**  Python is the dominant language.\n    *   **Data Structures and Algorithms:** Understanding the efficiency of different appro

# Routing Example
The `route` function allows us to route a query to the most appropriate model call based on a selector prompt.

In [9]:
query = {
    "prompt": """I'm having trouble understanding some calculus concepts. Can you help explain them or provide some resources?""",
    "model": model,
    "provider": provider,
    "system_prompt": system_prompt,
    "functions": [str.lower]
}

routes = {
    "math_tutor": {
        "prompt": """You are a math tutor. Follow these guidelines:
        1. Always start with "Math Tutor Response:"
        2. Acknowledge the specific math topic
        3. Provide clear explanations and examples
        4. Suggest additional resources or exercises
        5. Encourage the student to ask further questions

        Maintain a supportive and encouraging tone.
        """,
        "model": model,
        "provider": provider,
        "system_prompt": system_prompt,
        "functions": [str.lower]
    },
    "science_tutor": {
        "prompt": """You are a science tutor. Follow these guidelines:
        1. Always start with "Science Tutor Response:"
        2. Acknowledge the specific science topic
        3. Provide clear explanations and examples
        4. Suggest additional resources or experiments
        5. Encourage the student to ask further questions

        Maintain a supportive and encouraging tone.
        """,
        "model": model2,
        "provider": provider2,
        "system_prompt": system_prompt,
        "functions": [str.lower]
    },
    "english_tutor": {
        "prompt": """You are an English tutor. Follow these guidelines:
        1. Always start with "English Tutor Response:"
        2. Acknowledge the specific English topic
        3. Provide clear explanations and examples
        4. Suggest additional resources or exercises
        5. Encourage the student to ask further questions

        Maintain a supportive and encouraging tone.
        """,
        "model": model,
        "provider": provider,
        "system_prompt": system_prompt,
        "functions": [str.lower]
    },
    "history_tutor": {
        "prompt": """You are a history tutor. Follow these guidelines:
        1. Always start with "History Tutor Response:"
        2. Acknowledge the specific history topic
        3. Provide clear explanations and examples
        4. Suggest additional resources or readings
        5. Encourage the student to ask further questions

        Maintain a supportive and encouraging tone.
        """,
        "model": model2,
        "provider": provider2,
        "system_prompt": system_prompt,
        "functions": [str.lower]
    }
}

route_prompt = "Analyze the input and select the most appropriate tutor."

reasoning_prompt = ("Brief explanation of why this question should be routed to a specific tutor. "
                    "Consider key terms, user intent, and urgency level.")

In [10]:
# Synchronous routing
route_result = route(query=query, reasoning_prompt=reasoning_prompt, route_prompt=route_prompt, routes=routes)


Available routes: ['math_tutor', 'science_tutor', 'english_tutor', 'history_tutor']
Response: dict_keys(['candidates', 'create_time', 'response_id', 'model_version', 'prompt_feedback', 'usage_metadata', 'automatic_function_calling_history', 'parsed'])
Model: gemini-2.0-flash
Usage: cached_content_token_count=None candidates_token_count=98 prompt_token_count=134 total_token_count=232

Routing Analysis:
Reasoning: 
The user is struggling with calculus, a topic within mathematics. The question explicitly asks for help understanding concepts, indicating a need for mathematical explanations and resources.

Selection: math_tutor
Model: google-gemini-2.0-flash
Response: dict_keys(['candidates', 'create_time', 'response_id', 'model_version', 'prompt_feedback', 'usage_metadata', 'automatic_function_calling_history', 'parsed'])
Model: gemini-2.0-flash
Usage: cached_content_token_count=None candidates_token_count=215 prompt_token_count=100 total_token_count=315
Result: math tutor response:
Hi th

# Asynchronous Examples
We can also use the asynchronous versions of these functions.

In [11]:
# Define the asynchronous chaining function
async def async_chaining_example():
    print(f"Asynchronous Chaining Started...")
    async_chain_result = await achain(query=query, prompts=prompts)
    print(f"Asynchronous Chaining Result: {async_chain_result}")

# Define the asynchronous parallelization function
async def async_parallelization_example():
    print(f"Asynchronous Parallelization Started...")
    async_parallel_results = await aparallel(query=query, prompts=prompts)
    print(f"Asynchronous Parallelization Results: {async_parallel_results}")

# Define the asynchronous routing function
async def async_routing_example():
    print(f"Asynchronous Routing Started...")
    async_route_result = await aroute(query=query, reasoning_prompt=reasoning_prompt, route_prompt=route_prompt, routes=routes)
    print(f"Asynchronous Routing Result: {async_route_result}")

In [12]:
# Run the asynchronous chaining example
await async_chaining_example()

Asynchronous Chaining Started...
Model: google-gemini-2.0-flash

Step 1: what is the best way to learn a complex technical concept in deep learning?
Response: dict_keys(['candidates', 'create_time', 'response_id', 'model_version', 'prompt_feedback', 'usage_metadata', 'automatic_function_calling_history', 'parsed'])
Model: gemini-2.0-flash
Usage: cached_content_token_count=None candidates_token_count=1410 prompt_token_count=94 total_token_count=1504

Result: Okay, let's break down the best way to learn a complex technical concept in deep learning. Deep learning can be daunting, but with the right approach, you can master even the most challenging ideas.

**1. Identify the Core Concept and its Prerequisites:**

*   **Isolate the Specific Concept:** Vague goals like "understanding transformers" are too broad. Instead, focus on a specific part, like "attention mechanisms" or "positional encodings."
*   **Map Out Prerequisites:**  Deep learning concepts build on each other. What background 

In [13]:
# Run the asynchronous parallelization example
await async_parallelization_example()

Asynchronous Parallelization Started...
Response: dict_keys(['candidates', 'create_time', 'response_id', 'model_version', 'prompt_feedback', 'usage_metadata', 'automatic_function_calling_history', 'parsed'])
Model: gemini-2.0-flash
Usage: cached_content_token_count=None candidates_token_count=816 prompt_token_count=90 total_token_count=906
Response: dict_keys(['candidates', 'create_time', 'response_id', 'model_version', 'prompt_feedback', 'usage_metadata', 'automatic_function_calling_history', 'parsed'])
Model: gemini-2.0-flash
Usage: cached_content_token_count=None candidates_token_count=1539 prompt_token_count=94 total_token_count=1633
Response: dict_keys(['model', 'created_at', 'done', 'done_reason', 'total_duration', 'load_duration', 'prompt_eval_count', 'prompt_eval_duration', 'eval_count', 'eval_duration', 'response', 'context'])
Using model: deepseek-r1:1.5b
Response: dict_keys(['model', 'created_at', 'done', 'done_reason', 'total_duration', 'load_duration', 'prompt_eval_count',

In [14]:
# Run the asynchronous routing example
await async_routing_example()

Asynchronous Routing Started...

Available routes: ['math_tutor', 'science_tutor', 'english_tutor', 'history_tutor']
Response: dict_keys(['candidates', 'create_time', 'response_id', 'model_version', 'prompt_feedback', 'usage_metadata', 'automatic_function_calling_history', 'parsed'])
Model: gemini-2.0-flash
Usage: cached_content_token_count=None candidates_token_count=95 prompt_token_count=134 total_token_count=229

Routing Analysis:
Reasoning: 
The user's question explicitly mentions "calculus concepts," which is a branch of mathematics. The user is asking for help with understanding these concepts, which clearly falls under the purview of a math tutor.

Selection: math_tutor
Model: google-gemini-2.0-flash
Response: dict_keys(['candidates', 'create_time', 'response_id', 'model_version', 'prompt_feedback', 'usage_metadata', 'automatic_function_calling_history', 'parsed'])
Model: gemini-2.0-flash
Usage: cached_content_token_count=None candidates_token_count=276 prompt_token_count=100 to